## **Professor**: Luiz Fernando
## **Aluno**: Pedro Lucas Fernandes Carvalho

## **Descrição do notebook**: configurar a aquisição, limpeza e transformação de datasets da ONS, INMET e Nasa Power API


In [2]:
import pandas as pd

def filter_ons_csv(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process a monthly csv file from ONS and return a dataframe containing only data from photovoltaic sources
    
    Parameters
    ----------
    df: .csv file
        A CSV file from ONS containing power generation for a complete month
    
    Returns
    -------
    df: pd.dataframe 
        A cleaned monthly dataframe file from ONS containing only data from PV
    
    
    """
    import os

    df = pd.read_csv(df, sep = ";")   
    df = df[df['nom_tipousina'] == 'FOTOVOLTAICA']

    return df

In [3]:
import pandas as pd

def concatenate_ons_csv() -> pd.DataFrame:
    """
    Process all monthly csv file from ONS and return a yearly dataframe with all months data
    
    Parameters
    ----------

    Returns
    -------
    df: pd.DataFrame
        A yearly dataframe from ONS containing only data from PV
    """
   
    import os
    import glob

    filepath = os.getcwd() + '/datasets ons/'

    processed_dfs = []

    for file in glob.glob(f"{filepath}/*.csv"):
        df_cleaned = filter_ons_csv(file)
        processed_dfs.append(df_cleaned)

    yearly_df = pd.concat(processed_dfs, ignore_index=True)
    yearly_df.sort_values(by=['din_instante'], inplace = True)

    yearly_df = yearly_df.reset_index(drop = True)

    return yearly_df



In [4]:
def info_yearly_ons(df: pd.DataFrame):
    """
    Receives the yearly ONS dataframe and returns the description of it, and also #usines, #nulls and #NaN
    
    Parameters
    ----------
    df: pd.DataFrame
        A yearly dataframe from ONS containing only data from PV

    Returns
    ------- 
    """

    print(df.info())
    print(f"Quantidade de usinas fotovoltaicas no dataframe: {len(df['nom_usina'].unique())}" )
    print(f'Quantidade de valores nulos no DataFrame: {df.isnull().sum().sum()}')
    print(f'Quantidade de valores NaN no DataFrame: {df.isna().sum().sum()}')

In [5]:
info_yearly_ons(concatenate_ons_csv())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941472 entries, 0 to 941471
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   din_instante            941472 non-null  object 
 1   id_subsistema           941472 non-null  object 
 2   nom_subsistema          941472 non-null  object 
 3   id_estado               941472 non-null  object 
 4   nom_estado              941472 non-null  object 
 5   cod_modalidadeoperacao  941472 non-null  object 
 6   nom_tipousina           941472 non-null  object 
 7   nom_tipocombustivel     941472 non-null  object 
 8   nom_usina               941472 non-null  object 
 9   id_ons                  941472 non-null  object 
 10  ceg                     941472 non-null  object 
 11  val_geracao             940848 non-null  float64
dtypes: float64(1), object(11)
memory usage: 86.2+ MB
None
Quantidade de usinas fotovoltaicas no dataframe: 115
Quantidade de valores

In [6]:
df = concatenate_ons_csv()
df.head()

,din_instante,id_subsistema,nom_subsistema,id_estado,nom_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,id_ons,ceg,val_geracao
0,2024-01-01 00:00:00,N,NORTE,AC,ACRE,Pequenas Usinas (MMGD),FOTOVOLTAICA,Fotovoltaica,PQU ACRAC FOTOV MMGD,PQU_ACRAC_GD,-,0.000
1,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Janaúba,CJU_MGJAN,-,0.679
2,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Helio Valgas,CJU_MGVZPQ,-,0.576
3,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Araxá,CJU_MGARX,-,0.000
4,2024-01-01 00:00:00,SE,SUDESTE,GO,GOIAS,Pequenas Usinas (MMGD),FOTOVOLTAICA,Fotovoltaica,PQU DFGO FOTOV MMGD,PQU_DFGO_GD,-,0.000


In [7]:
df['cod_modalidadeoperacao'].unique()

array(['Pequenas Usinas (MMGD)', 'Conjunto de Usinas',
       'Pequenas Usinas (Tipo III)', 'TIPO II-B'], dtype=object)

In [8]:
def most_common_count(df) -> tuple:
    """
    Receives the yearly ONS dataframe and returns a tuple with the #entries for each usine and the maximum #entries in the dataset
    
    Parameters
    ----------
    df: pd.DataFrame
        A yearly dataframe from ONS containing only data from PV

    Returns
    ------- 
    tuple: (counts,most_common_count)
    """

    counts = df.groupby('nom_usina').size()
    most_common_count = counts.value_counts().idxmax()
    return counts,most_common_count

In [9]:
def identify_missing_data_usines(df: pd.DataFrame) -> pd.DataFrame:
    """
    Receives the yearly ONS dataframe and returns a dataframe containing the names and #of entries of usines without a complete year time series
    
    Parameters
    ----------
    df: pd.DataFrame
        A yearly dataframe from ONS containing only data from PV

    Returns
    ------- 
    df: pd.DataFrame
    """

    usine_counts, most_common_usine_count = most_common_count(df)
    usines_missing_data = usine_counts[usine_counts != most_common_usine_count].reset_index()

    return usines_missing_data

def identify_complete_data_usines(df: pd.DataFrame) -> pd.DataFrame:
    """
    Receives the yearly ONS dataframe and returns a dataframe containing the names and #of entries of usines with a complete year time series
    
    Parameters
    ----------
    df: pd.DataFrame
        A yearly dataframe from ONS containing only data from PV

    Returns
    ------- 
    df: pd.DataFrame
    """

    usine_counts, most_common_usine_count = most_common_count(df)
    usines_complete_data  = usine_counts[usine_counts == most_common_usine_count].reset_index()

    return usines_complete_data

In [10]:
complete_df = identify_complete_data_usines(df)
complete_df['nom_usina']

0                                Assú V
1                            Conj. Alex
2                           Conj. Araxá
3                             Conj. BJL
4                        Conj. Belmonte
                    ...                
87                  PQU SPMG FOTOV MMGD
88                  PQU SPMS FOTOV MMGD
89        PQU SPSP FOTOV - CARGA GLOBAL
90                  PQU SPSP FOTOV MMGD
91    PQU TO NORTE FOTOV - CARGA GLOBAL
Name: nom_usina, Length: 92, dtype: object

In [11]:
identify_missing_data_usines(df)

,nom_usina,0
0,Conj. Arinos 2 500 kV,8064
1,Conj. Banabuiu,8712
2,Conj. Boa Sorte,7104
3,Conj. Bom Nome,6072
4,Conj. Fotov. Acu III 230KV,8064
5,Conj. Fotov. Simplice,960
6,Conj. Fotovoltaico Abaiara 230 kV,8064
7,Conj. Fotovoltaico Barreiras II 500 kV,4752
8,Conj. Fotovoltaico Gilbués II 500 kV,2496
9,Conj. Jaiba V,7512


In [12]:
df[df['nom_usina'] == 'Conj. Fotovoltaico Abaiara 230 kV'].head(5)

,din_instante,id_subsistema,nom_subsistema,id_estado,nom_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,id_ons,ceg,val_geracao
69285,2024-01-31 00:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Fotovoltaico Abaiara 230 kV,CJU_CEAB2,-,0.523
69368,2024-01-31 01:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Fotovoltaico Abaiara 230 kV,CJU_CEAB2,-,0.523
69487,2024-01-31 02:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Fotovoltaico Abaiara 230 kV,CJU_CEAB2,-,0.523
69589,2024-01-31 03:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Fotovoltaico Abaiara 230 kV,CJU_CEAB2,-,0.523
69684,2024-01-31 04:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Fotovoltaico Abaiara 230 kV,CJU_CEAB2,-,0.523


In [13]:
def get_location_df()-> pd.DataFrame:
    """
    Opens a dataframe with the name of active usines - with disponible location or without mmgd
    
    --------

    Returns
    ------- 
    df: pd.DataFrame
    """
    
    import os

    filepath = os.getcwd() 
    location_df = pd.read_csv(filepath +'/localizacao usinas.csv', sep = ";")
    return location_df
    

In [14]:
def get_active_usines(df: pd.DataFrame) -> pd.DataFrame:
    """
    Receives a dataframe - supposed to use the yearly dataframe - and returns the dataframe with location for each usine with disponible location
    
    --------
    Parameters
    df: pd.DataFrame
        A yearly dataframe from ONS containing only data from PV

    Returns
    ------- 
    df: pd.DataFrame
        A yearly dataframe from ONS containing PV only data, geolocated
    """
    location_df = get_location_df()


    mask = df['nom_usina'].isin(location_df['nom_usina'])
    df_active_usines = df[mask].copy()

    df_active_usines = pd.merge(df_active_usines, location_df, how="inner", on="nom_usina")
    return df_active_usines

In [15]:
df_active_usines = get_active_usines(df)

In [1]:
import requests
import os
import time
import concurrent.futures
import random
import numpy as np

# Define the output directory
output = r"./output_brazil_grid"
os.makedirs(output, exist_ok=True)

# Define the base URL for the API request (Hourly data for the year 2024)
base_url = r"https://power.larc.nasa.gov/api/temporal/hourly/point?parameters={parameters}&community=RE&longitude={longitude}&latitude={latitude}&start=20240101&end=20241231&format=CSV"

# List of renewable energy parameters available in the NASA POWER API
parameters = [
    "ALLSKY_SFC_SW_DWN",  # Surface Downward Shortwave Radiation (W/m²)
    "T2M",  # 2m Temperature (°C)
    "RH2M",  # 2m Relative Humidity (%)
    "WS10M",  # 10m Wind Speed (m/s)
    #"PRECTOT",  # Total Precipitation (mm)
    #"SH2O",  # Surface Humidity (g/m²)
    #"SRAD",  # Solar Radiation (W/m²)
    #"CLRSKY_SFC_SW_DWN",  # Clear Sky Surface Downward Shortwave Radiation (W/m²)
    "ALLSKY_KT" #,  # Surface Pressure (Pa)
    #"WSPD10M",  # Wind Speed at 10m (m/s)
]

# Define the grid of latitudes and longitudes (0.5 x 0.5 resolution for Brazil)
latitudes = list(np.arange(-34.0, 6.0, 0.5))      # From -34.0 to 6.0
longitudes = list(np.arange(-74.0, -34.0, 0.5))   # From -74.0 to -34.0

# Function to make API request and save the data for a given latitude and longitude
def fetch_data_for_location(latitude, longitude, param_list):
    # Construct the API request URL for the current grid point
    params = ','.join(param_list)
    api_request_url = base_url.format(longitude=longitude, latitude=latitude, parameters=params)

    retries = 1  # Number of retries in case of error
    backoff_factor = 2  # Exponential backoff factor

    for attempt in range(retries):
        try:
            # Make the API request with a timeout of 30 seconds
            response = requests.get(api_request_url, verify=True, timeout=30.0)
            
            # Check if the response status is OK (200)
            if response.status_code == 200:
                # Define the output CSV file path
                filename = f"location_{latitude}_{longitude}_2024.csv"
                filepath = os.path.join(output, filename)

                # Write the data to a CSV file
                with open(filepath, 'wb') as csvfile:
                    csvfile.write(response.content)  # Save the CSV content directly

                print(f"Data for location ({latitude}, {longitude}) saved to {filepath}")
                break  # Exit the loop if the request was successful
            else:
                print(f"Error {response.status_code}: Could not fetch data for location ({latitude}, {longitude})")
                time.sleep(10)  # Wait before retrying
        except requests.exceptions.RequestException as e:
            print(f"Request error for location ({latitude}, {longitude}): {e}")
            if attempt < retries - 1:
                time.sleep(backoff_factor ** attempt)  # Exponential backoff
            else:
                print(f"Failed after {retries} attempts for location ({latitude}, {longitude})")
                break

# Create a list of all latitude and longitude pairs
locations = [(lat, lon) for lat in latitudes for lon in longitudes]

# Function to handle batching of parameters (to stay within the limit of 10 parameters per call)
def chunk_parameters(parameter_list, chunk_size=10):
    for i in range(0, len(parameter_list), chunk_size):
        yield parameter_list[i:i + chunk_size]

# Using ThreadPoolExecutor to run the API calls in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Loop through all latitude and longitude locations
    future_to_location = {}
    for lat, lon in locations:
        for param_chunk in chunk_parameters(parameters):
            # Submit each API call in parallel
            future = executor.submit(fetch_data_for_location, lat, lon, param_chunk)
            future_to_location[future] = (lat, lon)

    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(future_to_location):
        location = future_to_location[future]
        try:
            future.result()  # If any exception occurred, it will be raised here
        except Exception as e:
            print(f"Exception for {location}: {e}")

        # Delay to avoid hitting API rate limits
        time.sleep(2)  # sleep 1 second between requests to avoid overloading the server


Data for location (-34.0, -73.5) saved to ./output_brazil_grid/location_-34.0_-73.5_2024.csv
Data for location (-34.0, -71.0) saved to ./output_brazil_grid/location_-34.0_-71.0_2024.csv
Data for location (-34.0, -74.0) saved to ./output_brazil_grid/location_-34.0_-74.0_2024.csv
Data for location (-34.0, -73.0) saved to ./output_brazil_grid/location_-34.0_-73.0_2024.csv
Data for location (-34.0, -70.5) saved to ./output_brazil_grid/location_-34.0_-70.5_2024.csv
Data for location (-34.0, -72.5) saved to ./output_brazil_grid/location_-34.0_-72.5_2024.csv
Data for location (-34.0, -72.0) saved to ./output_brazil_grid/location_-34.0_-72.0_2024.csv
Data for location (-34.0, -69.5) saved to ./output_brazil_grid/location_-34.0_-69.5_2024.csv
Data for location (-34.0, -70.0) saved to ./output_brazil_grid/location_-34.0_-70.0_2024.csv
Data for location (-34.0, -71.5) saved to ./output_brazil_grid/location_-34.0_-71.5_2024.csv
Data for location (-34.0, -65.5) saved to ./output_brazil_grid/locatio

KeyboardInterrupt: 

In [3]:
import pandas as pd

def filter_nasa_csv(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process a yearly csv file from ONS for a given location and return a cleaned dataframe for this location
    
    Parameters
    ----------
    df: .csv file
        A CSV file from NASA Power API containing climate parameters
    
    Returns
    -------
    df: pd.dataframe 
        A cleaned dataframe file from NASA
    
    
    """
    import os
    import re

    filepath = os.getcwd() + '/output_brazil_grid/'
    filename = df

    df = pd.read_csv(df, skiprows = 12, sep = ",")
    df.columns = df.iloc[0]
    df = df[1:].reset_index()
    df.columns.name = None
    df = df.rename(columns={"level_0": "YEAR", "level_1": "MO", "level_2": "DY", "level_3": "HR", "level_4": "ALLSKY_SFC_SW_DWN", "level_5": "T2M", "level_6": " RH2M", "level_7": "WS10M"})

    match = re.search(r"location_([-+]?[0-9]*\.?[0-9]+)_([-+]?[0-9]*\.?[0-9]+)_\d{4}\.csv", filename)
    if match:
        lat = float(match.group(1))
        lon = float(match.group(2))

    df['latitude'] = lat
    df['longitude'] = lon
    #df['location'] = f"{lat},{lon}"

    return df

In [ ]:
import pandas as pd

def concatenate_nasa_csv() -> pd.DataFrame:
    """
    Process all csv file from NASA Power API and return a yearly dataframe
    
    Parameters
    ----------

    Returns
    -------
    df: pd.DataFrame
        A yearly dataframe from NASA containing all the data for the grid
    """
   
    import os
    import glob

    filepath = os.getcwd() + '/output_brazil_grid/'

    processed_dfs = []

    for file in glob.glob(f"{filepath}/*.csv"):
        df_cleaned = filter_nasa_csv(file)
        processed_dfs.append(df_cleaned)
        print('File' + file + 'filtered and appended')

    yearly_df = pd.concat(processed_dfs, ignore_index=True)
    #yearly_df.sort_values(by=['din_instante'], inplace = True)

    yearly_df = yearly_df.reset_index(drop = True)

    return yearly_df



: 

In [ ]:
import pandas as pd #works
import os
import glob

def concatenate_nasa_csv_streamed(output_path: str = "nasa_grid_data.parquet"):
    """
    Processes and streams each CSV file from NASA Power API into a Parquet file on disk.

    Parameters
    ----------
    output_path : str
        File path to save the concatenated DataFrame.

    Returns
    -------
    None
    """
    filepath = os.getcwd() + '/output_brazil_grid/'
    all_files = glob.glob(f"{filepath}/*.csv")

    # Delete old output if exists
    if os.path.exists(output_path):
        os.remove(output_path)

    for i, file in enumerate(all_files):
        df_cleaned = filter_nasa_csv(file)
        
        df_cleaned.to_parquet(
            output_path, 
            engine="fastparquet",  # or "pyarrow"
            compression="snappy",
            index=False,
            append=(i != 0),  # Append after the first file
        )
        print(f"[{i+1}/{len(all_files)}] Written {file}")

    print("✅ All files streamed to", output_path)


In [6]:
concatenate_nasa_csv_streamed()

[1/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-12.5_-61.5_2024.csv
[2/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-22.5_-41.0_2024.csv
[3/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_1.0_-63.5_2024.csv
[4/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-26.5_-59.5_2024.csv
[5/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_2.0_-56.5_2024.csv
[6/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-21.5_-40.0_2024.csv
[7/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-25.5_-64.0_2024.csv
[8/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-15.0_-71.0_2024.csv
[9/6478] Written /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_2.5_-66.0_2024.csv
[10/6478] Written /home/pedro/Área de Traba

In [1]:
import pandas as pd
df_test = pd.read_parquet("nasa_grid_data.parquet", engine = 'pyarrow')

In [3]:
df_test.shape

(56902752, 11)

In [ ]:
df_test = concatenate_nasa_csv()    

[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-12.5_-61.5_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-22.5_-41.0_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_1.0_-63.5_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-26.5_-59.5_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_2.0_-56.5_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-21.5_-40.0_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-25.5_-64.0_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_-15.0_-71.0_2024.csv
[Batch 1] Processed /home/pedro/Área de Trabalho/tcc/codigos/output_brazil_grid/location_2.5_-66.0_2024.csv
[Batch 1] Proces

In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56902752 entries, 0 to 56902751
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   YEAR               object 
 1   MO                 object 
 2   DY                 object 
 3   HR                 object 
 4   ALLSKY_SFC_SW_DWN  object 
 5   T2M                object 
 6    RH2M              object 
 7   WS10M              object 
 8   ALLSKY_KT          object 
 9   latitude           float64
 10  longitude          float64
dtypes: float64(2), object(9)
memory usage: 4.7+ GB


In [16]:
df_test.head(20)

,YEAR,MO,DY,HR,ALLSKY_SFC_SW_DWN,T2M,RH2M,WS10M,ALLSKY_KT,latitude,longitude
0,2024,1,1,0,0.0,26.82,75.33,0.47,-999.0,-12.5,-61.5
1,2024,1,1,1,0.0,26.35,78.06,0.38,-999.0,-12.5,-61.5
2,2024,1,1,2,0.0,25.92,81.63,0.27,-999.0,-12.5,-61.5
3,2024,1,1,3,0.0,25.62,83.73,0.18,-999.0,-12.5,-61.5
4,2024,1,1,4,0.0,25.34,89.15,0.22,-999.0,-12.5,-61.5
5,2024,1,1,5,0.0,25.0,85.66,0.65,-999.0,-12.5,-61.5
6,2024,1,1,6,46.72,25.48,84.58,0.59,0.19,-12.5,-61.5
7,2024,1,1,7,132.12,27.88,69.56,0.6,0.24,-12.5,-61.5
8,2024,1,1,8,262.4,29.69,62.65,0.75,0.31,-12.5,-61.5
9,2024,1,1,9,375.75,30.49,60.48,0.7,0.35,-12.5,-61.5


In [17]:
import pandas as pd

# Define chunk size
chunk_size = 500_000  # or tune this for your system

# Preallocate datetime column
datetime_col = []

for start in range(0, len(df_test), chunk_size):
    end = start + chunk_size
    chunk = df_test.iloc[start:end].copy()

    # Convert columns to strings (in memory-efficient slice)
    year = chunk['YEAR'].astype(str)
    mo = chunk['MO'].astype(str).str.zfill(2)
    dy = chunk['DY'].astype(str).str.zfill(2)
    hr = chunk['HR'].astype(str).str.zfill(2)

    # Combine into datetime string and parse
    dt_str = year + '-' + mo + '-' + dy + ' ' + hr + ':00:00'
    dt = pd.to_datetime(dt_str, format='%Y-%m-%d %H:%M:%S')

    datetime_col.append(dt)

# Concatenate all chunks
df_test['datetime'] = pd.concat(datetime_col, ignore_index=True)


In [4]:
df_test.tail()

,YEAR,MO,DY,HR,ALLSKY_SFC_SW_DWN,T2M,RH2M,WS10M,ALLSKY_KT,latitude,longitude,datetime
56902747,2024,12,31,19,-999.0,25.28,98.97,0.79,-999.0,-4.0,-71.0,2024-12-31 19:00:00
56902748,2024,12,31,20,-999.0,25.02,99.23,0.89,-999.0,-4.0,-71.0,2024-12-31 20:00:00
56902749,2024,12,31,21,-999.0,24.77,99.5,1.08,-999.0,-4.0,-71.0,2024-12-31 21:00:00
56902750,2024,12,31,22,-999.0,24.61,99.25,1.23,-999.0,-4.0,-71.0,2024-12-31 22:00:00
56902751,2024,12,31,23,-999.0,24.64,98.12,1.13,-999.0,-4.0,-71.0,2024-12-31 23:00:00


In [ ]:
df_test.rename(columns={"datetime": "din_instante"})


,YEAR,MO,DY,HR,ALLSKY_SFC_SW_DWN,T2M,RH2M,WS10M,ALLSKY_KT,latitude,longitude,din_instante
0,2024,1,1,0,0.0,26.82,75.33,0.47,-999.0,-12.5,-61.5,2024-01-01 00:00:00
1,2024,1,1,1,0.0,26.35,78.06,0.38,-999.0,-12.5,-61.5,2024-01-01 01:00:00
2,2024,1,1,2,0.0,25.92,81.63,0.27,-999.0,-12.5,-61.5,2024-01-01 02:00:00
3,2024,1,1,3,0.0,25.62,83.73,0.18,-999.0,-12.5,-61.5,2024-01-01 03:00:00
4,2024,1,1,4,0.0,25.34,89.15,0.22,-999.0,-12.5,-61.5,2024-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
56902747,2024,12,31,19,-999.0,25.28,98.97,0.79,-999.0,-4.0,-71.0,2024-12-31 19:00:00
56902748,2024,12,31,20,-999.0,25.02,99.23,0.89,-999.0,-4.0,-71.0,2024-12-31 20:00:00
56902749,2024,12,31,21,-999.0,24.77,99.5,1.08,-999.0,-4.0,-71.0,2024-12-31 21:00:00
56902750,2024,12,31,22,-999.0,24.61,99.25,1.23,-999.0,-4.0,-71.0,2024-12-31 22:00:00


: 

In [ ]:
df_test.rename(columns={"latitude": "closest_latitude"})


In [ ]:
df_test.rename(columns={"longitude": "closest_latitude"})

In [ ]:
df_one_entry = df_active_usines.drop_duplicates('nom_usina').copy()

In [40]:
df_one_entry.tail()

,din_instante,id_subsistema,nom_subsistema,id_estado,nom_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,id_ons,ceg,val_geracao,latitude,longitude,closest_latitude,closest_longitude,distance_km
333792,2024-01-01 00:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Serra do Mato,CJU_CEFSEM,-,0.0,-3.305928,-39.358492,-3.5,-39.5,26.690958
342576,2024-01-01 00:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Sol do Futuro,CJU_CESDF,-,0.0,-3.979306,-38.388869,-4.0,-38.5,12.540185
351360,2024-01-01 00:00:00,NE,NORDESTE,CE,CEARA,TIPO II-B,FOTOVOLTAICA,Fotovoltaica,Flor de Mandacaru,CEFFDM,UFV.RS.CE.047255-7.01,0.0,-4.779134,-37.274876,-5.0,-37.5,35.003316
360144,2024-01-01 00:00:00,NE,NORDESTE,CE,CEARA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Alex,CJU_CEUFAL,-,0.0,-5.253291,-37.943225,-5.5,-38.0,28.143611
368928,2024-01-01 00:00:00,NE,NORDESTE,PB,PARAIBA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Luzia,CJU_PBLZA,-,0.0,-6.841888,-36.944959,-7.0,-37.0,18.601509


In [ ]:
import numpy as np
import pandas as pd

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat/2.0)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c  # distance in kilometers

# We'll loop through df_one_entry and find closest in df_test using haversine
closest_lats = []
closest_lons = []
closest_dists = []

for _, row in df_one_entry.iterrows():
    lat1, lon1 = row['latitude'], row['longitude']
    lat2_all = df_test['latitude'].values
    lon2_all = df_test['longitude'].values

    # Compute distances to all df_test points
    dists = haversine(lat1, lon1, lat2_all, lon2_all)

    # Find the index of the minimum distance
    idx_min = np.argmin(dists)

    # Store closest lat, lon and distance
    closest_lats.append(df_test.iloc[idx_min]['latitude'])
    closest_lons.append(df_test.iloc[idx_min]['longitude'])
    closest_dists.append(dists[idx_min])

# Add results to df_one_entry
df_one_entry['closest_latitude'] = closest_lats
df_one_entry['closest_longitude'] = closest_lons
df_one_entry['distance_km'] = closest_dists

In [22]:
print("Unique latitudes in df_test:", sorted(df_test['latitude'].unique()))
print("Unique longitudes in df_test:", sorted(df_test['longitude'].unique()))
print("Total grid points:", len(df_test))


Unique latitudes in df_test: [-34.0, -33.5, -33.0, -32.5, -32.0, -31.5, -31.0, -30.5, -30.0, -29.5, -29.0, -28.5, -28.0, -27.5, -27.0, -26.5, -26.0, -25.5, -25.0, -24.5, -24.0, -23.5, -23.0, -22.5, -22.0, -21.5, -21.0, -20.5, -20.0, -19.5, -19.0, -18.5, -18.0, -17.5, -17.0, -16.5, -16.0, -15.5, -15.0, -14.5, -14.0, -13.5, -13.0, -12.5, -12.0, -11.5, -11.0, -10.5, -10.0, -9.5, -9.0, -8.5, -8.0, -7.5, -7.0, -6.5, -6.0, -5.5, -5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0]
Unique longitudes in df_test: [-74.0, -73.5, -73.0, -72.5, -72.0, -71.5, -71.0, -70.5, -70.0, -69.5, -69.0, -68.5, -68.0, -67.5, -67.0, -66.5, -66.0, -65.5, -65.0, -64.5, -64.0, -63.5, -63.0, -62.5, -62.0, -61.5, -61.0, -60.5, -60.0, -59.5, -59.0, -58.5, -58.0, -57.5, -57.0, -56.5, -56.0, -55.5, -55.0, -54.5, -54.0, -53.5, -53.0, -52.5, -52.0, -51.5, -51.0, -50.5, -50.0, -49.5, -49.0, -48.5, -48.0, -47.5, -47.0, -46.5, -46.0, -45.5, -45.0, -44.

In [23]:
df_one_entry.head(10)

,din_instante,id_subsistema,nom_subsistema,id_estado,nom_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,id_ons,ceg,val_geracao,latitude,longitude,closest_latitude,closest_longitude,distance_km
0,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Janaúba,CJU_MGJAN,-,0.679,-15.930854,-43.531287,-16.0,-43.5,8.384708
8784,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Helio Valgas,CJU_MGVZPQ,-,0.576,-17.527384,-44.627941,-17.5,-44.5,13.904426
17568,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Araxá,CJU_MGARX,-,0.000,-19.754675,-47.024776,-20.0,-47.0,27.401650
26352,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Sol do Cerrado,CJU_MGSDC,-,0.000,-15.381898,-43.781940,-15.5,-44.0,26.808714
35136,2024-01-01 00:00:00,NE,NORDESTE,RN,RIO GRANDE DO NORTE,TIPO II-B,FOTOVOLTAICA,Fotovoltaica,Assú V,RNFVAE,UFV.RS.RN.034184-3.01,0.000,-5.571501,-37.029055,-5.5,-37.0,8.576201
43920,2024-01-01 00:00:00,NE,NORDESTE,RN,RIO GRANDE DO NORTE,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Floresta,CJU_RNFLO,-,0.000,-4.962229,-36.910832,-5.0,-37.0,10.733370
52704,2024-01-01 00:00:00,NE,NORDESTE,RN,RIO GRANDE DO NORTE,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Serra do Mel C,CJU_RNSDMC,-,0.901,-5.146276,-37.007942,-5.0,-37.0,16.288967
61488,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Paracatu,CJU_MGUFPU,-,6.618,-17.219286,-47.083875,-17.0,-47.0,25.961689
70272,2024-01-01 00:00:00,SE,SUDESTE,MG,MINAS GERAIS,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Francisco Sá,CJU_MGFSA,-,0.000,-16.291438,-43.648061,-16.5,-43.5,28.058448
79056,2024-01-01 00:00:00,SE,SUDESTE,SP,SAO PAULO,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Pereira Barreto,CJU_SPPB,-,0.000,-20.611527,-51.285618,-20.5,-51.5,25.534080


In [21]:
df_active_usines_complete = df_active_usines.copy()

In [22]:
df_active_usines_complete = df_active_usines_complete.merge(
    df_one_entry[['nom_usina', 'closest_latitude','closest_longitude','distance_km']],
    on='nom_usina',
    how='left'
)

In [23]:
df_active_usines_complete.tail()

,din_instante,id_subsistema,nom_subsistema,id_estado,nom_estado,cod_modalidadeoperacao,nom_tipousina,nom_tipocombustivel,nom_usina,id_ons,ceg,val_geracao,latitude,longitude,closest_latitude,closest_longitude,distance_km
377707,2024-12-31 19:00:00,NE,NORDESTE,PB,PARAIBA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Luzia,CJU_PBLZA,-,0.0,-6.841888,-36.944959,-7.0,-37.0,18.601509
377708,2024-12-31 20:00:00,NE,NORDESTE,PB,PARAIBA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Luzia,CJU_PBLZA,-,0.0,-6.841888,-36.944959,-7.0,-37.0,18.601509
377709,2024-12-31 21:00:00,NE,NORDESTE,PB,PARAIBA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Luzia,CJU_PBLZA,-,0.0,-6.841888,-36.944959,-7.0,-37.0,18.601509
377710,2024-12-31 22:00:00,NE,NORDESTE,PB,PARAIBA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Luzia,CJU_PBLZA,-,0.0,-6.841888,-36.944959,-7.0,-37.0,18.601509
377711,2024-12-31 23:00:00,NE,NORDESTE,PB,PARAIBA,Conjunto de Usinas,FOTOVOLTAICA,Fotovoltaica,Conj. Luzia,CJU_PBLZA,-,0.0,-6.841888,-36.944959,-7.0,-37.0,18.601509


In [ ]:
import pandas as pd

# Create an empty list to collect processed chunks
merged_chunks = []

# Define chunk size
chunk_size = 100_000  # Adjust based on memory limits

# Iterate through chunks of df_active_usines_complete
for start in range(0, len(df_active_usines_complete), chunk_size):
    end = start + chunk_size
    chunk = df_active_usines_complete.iloc[start:end].copy()

    # Merge on the three keys: 'din_instante', 'closest_latitude', 'closes_longitude'
    merged_chunk = chunk.merge(
        df_test[['din_instante', 'latitude', 'longitude']], 
        on=['din_insante', 'closest_latitude', 'closest_longitude'],
        how='left'  # 'left' ensures we keep all rows from df_active_usines_complete
    )
    
    merged_chunks.append(merged_chunk)

# Concatenate all chunks back into a single DataFrame
df_active_usines_complete = pd.concat(merged_chunks, ignore_index=True)


KeyError: "['din_insante'] not in index"